# 6.3 Geospatial Analysis

### This script contains the following:
#### 1. Import data and libraries
#### 2. Data wrangling
#### 3. Data cleaning
#### 4. Plotting a choropleth
#### 5. Discussing the result

### 1. Import data and libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# propting matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
# Import ".json" file for the U.S. 

country_geo =r'C:\Users\mojis\Documents\11-2022-Gun violence Analysis\02-Data\Json folder\us-states.json'

In [4]:
# JSON file contents:

f = open(r'C:\Users\mojis\Documents\11-2022-Gun violence Analysis\02-Data\Json folder\us-states.json',)
  
# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}
{'type': 'Feature', 'id': 'AK', 'properties': {'name': 'Alaska'},

#### Data

In [5]:
#Creating path
path =r'C:\Users\mojis\Documents\11-2022-Gun violence Analysis'


In [6]:
#Import gun_violence data
df=pd.read_pickle(os.path.join(path,'02-Data', 'Prepared data', 'gun_violence_updated_clean.pkl'))

In [7]:
df.head()

incident_id       date           state city_or_county  n_killed  n_injured  \
0      461105 2013-01-01    Pennsylvania     Mckeesport         0          4   
1      460726 2013-01-01      California      Hawthorne         1          3   
2      478855 2013-01-01            Ohio         Lorain         1          3   
3      478925 2013-01-05        Colorado         Aurora         4          0   
4      478959 2013-01-07  North Carolina     Greensboro         2          2   

   latitude  longitude  n_guns_involved  year  month  monthday  weekday  \
0   40.3467   -79.8559              NaN  2013      1         1        1   
1   33.9090  -118.3330              NaN  2013      1         1        1   
2   41.4455   -82.1377              2.0  2013      1         1        1   
3   39.6518  -104.8020              NaN  2013      1         5        5   
4   36.1140   -79.9569              2.0  2013      1         7        0   

   n_shot  shot Category weekday_names  
0       4  Moderate Loss       Mondays  
1       4  Moderate Loss       Mondays  
2       4  Moderate Loss       Mondays  
3       4  Moderate Loss       Fridays  
4       4  Moderate Loss       Sundays

In [8]:
df.shape

(239677, 16)

### 2. Data wrangling

In [9]:
df.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'n_killed',
       'n_injured', 'latitude', 'longitude', 'n_guns_involved', 'year',
       'month', 'monthday', 'weekday', 'n_shot', 'shot Category',
       'weekday_names'],
      dtype='object')

No dummy columns

In [10]:
#create a subset to count number of incidents by every state

states_df = df['state'].value_counts()

statesdf = pd.DataFrame()
statesdf['state'] = states_df.index
statesdf['counts'] = states_df.values

In [11]:
statesdf

state  counts
0               Illinois   17556
1             California   16306
2                Florida   15029
3                  Texas   13577
4                   Ohio   10244
5               New York    9712
6           Pennsylvania    8929
7                Georgia    8925
8         North Carolina    8739
9              Louisiana    8103
10             Tennessee    7626
11        South Carolina    6939
12              Missouri    6631
13              Michigan    6136
14         Massachusetts    5981
15              Virginia    5949
16               Indiana    5852
17              Maryland    5798
18               Alabama    5471
19            New Jersey    5387
20             Wisconsin    4787
21              Kentucky    4157
22           Mississippi    3599
23              Oklahoma    3455
24            Washington    3434
25              Colorado    3201
26  District of Columbia    3195
27           Connecticut    3067
28              Arkansas    2842
29                  Iowa    2517
30             Minnesota    2408
31               Arizona    2328
32                Oregon    2286
33                Kansas    2145
34                Nevada    1952
35              Delaware    1685
36              Nebraska    1651
37            New Mexico    1645
38         West Virginia    1575
39                Alaska    1349
40                  Utah    1072
41         New Hampshire     964
42                 Maine     907
43          Rhode Island     895
44                 Idaho     661
45               Montana     638
46          North Dakota     573
47          South Dakota     544
48               Wyoming     494
49               Vermont     472
50                Hawaii     289

### 3. consistency checks

In [12]:
# Check for missing values

statesdf.isnull().sum()

state     0
counts    0
dtype: int64

##### duplicate check

In [13]:
# checking for duplicate in df2
dup = statesdf[statesdf.duplicated()]

In [14]:
dup.shape # no duplicate

(0, 2)

### 4. Plotting a choropleth

In [15]:
# data frame to plot

data_to_plot = statesdf
data_to_plot.head()

state  counts
0    Illinois   17556
1  California   16306
2     Florida   15029
3       Texas   13577
4        Ohio   10244

In [16]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot,
    columns = ['state', 'counts'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Gun violence Incidents").add_to(map)
folium.LayerControl().add_to(map)

map

The above map is showing number of gun violence incident by states in  the U.S.

In [17]:
map.save('plot_data.html')

### 5. Discussing the result

#### Discuss the results and what they mean in a markdown section.
- Does the analysis answer any of your existing research questions?
- Does the analysis lead you to any new research questions?

##### Question :- Which states has the highest rate of gun violence incidents in the US.?

From the above map and analysis, Illinoies State has the highest number of gun violence incidents reported in last 5 years, the number is equal to 17,556, followed by California with 16306 incidents, Florida with 15,029 incidents and Texas with 13,577 gun violence incidents